1. Import required packages
2. read csv files
3. merge join files

In [1]:
# importing python packages:
import pandas as pd
from decimal import Decimal
from sklearn.metrics import mean_absolute_error
from matplotlib.pyplot import hist
import seaborn as sns
from numpy import sum
from numpy import median
from numpy import mean
from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
from matplotlib.pyplot import hist
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
# loading files:
main_file="../input/neobank-transactions/2016-09-19_79351_training.csv"
main=pd.read_csv(main_file,parse_dates=['transaction_date'], dtype = {'mcc_group':'str','user_id': 'str','amount_n26_currency': 'int','transaction_type': 'str'})
transaction_type_file="../input/neobank-transactions/transaction_types.csv"
transaction_type=pd.read_csv(transaction_type_file, dtype='str')
mcc_group_file="../input/neobank-transactions/mcc_group_definition.csv"
mcc_group=pd.read_csv(mcc_group_file, dtype='str')
# handling null values:
main.columns = main.columns.str.strip()
main['mcc_group'] = main['mcc_group'].fillna('Unknown')
# data integration:
testing_training=pd.merge(left=main,right=transaction_type,how='left', left_on='transaction_type', right_on='type')
testing_training=pd.merge(left=testing_training,right=mcc_group,how='left', left_on='mcc_group', right_on='mcc_group')
testing_training=testing_training[['user_id','transaction_date','explanation_y','agent','direction','explanation_x','amount_n26_currency']]

1. create a delivered column for month of transaction
2. split to trainig and testing sets. Testing is for the month July and training is for Feb-May.
3. removing non-mutual users
4. #soring by user id:

In [1]:
#delivered column for month:
testing_training['month'] = testing_training['transaction_date'].dt.month
#split to trainig and testing sets:
testing=testing_training[testing_training['month']==7]
training=testing_training[testing_training['month']!=7]
#removing non-mutual users:
cond = ~training['user_id'].isin(testing['user_id'])
training=training.drop(training[cond].index)
cond = ~testing['user_id'].isin(training['user_id'])
testing=testing.drop(testing[cond].index)
#soring by user id:
testing =testing.sort_values(by=['user_id'], ascending=True)
training =training.sort_values(by=['user_id'], ascending=True)

1. Calculating average for training set
2. Comparing the average of training set with corresponding users in testing set
3. measuring accuracy using MAPE
4. visualization

In [1]:
#converting direction rows to columns:
testing_user=testing.pivot_table( index = ['user_id'] , columns=['direction'], values = 'amount_n26_currency', aggfunc='mean').fillna(0)
testing_user=testing_user.reset_index()
training_user=training.pivot_table( index = ['user_id'] , columns=['direction'], values = 'amount_n26_currency', aggfunc='mean').fillna(0)
training_user=training_user.reset_index()
#instantiating:
mape_total_out=0
mape_total_in=0
total_AE_out={}
total_AE_in={}
# avoiding division by zero error using temporary values:
training_user.loc[training_user['In'] ==0, 'In'] =0.46395169417715465
training_user.loc[training_user['Out'] ==0, 'Out'] = 0.6283420872431762
testing_user.loc[testing_user['In'] ==0, 'In'] =0.46395169417715465
testing_user.loc[testing_user['Out'] ==0, 'Out'] = 0.6283420872431762
#comparing Comparing the average of training set with corresponding users in testing set:
for i in range(0, len(training_user.index)):
        if float(training_user.values[i,2])==float(testing_user.values[i,2]):
            mape_out=0
        else:
            mape_out=abs((float(training_user.values[i,2])-float(testing_user.values[i,2]))/float(training_user.values[i,2]))*100
        total_AE_out[training_user.values[i,0]]=mape_out
        mape_total_out=mape_total_out+mape_out
        if float(training_user.values[i,1])==float(testing_user.values[i,1]):
            mape_in=0
        else:
            mape_in=abs((float(training_user.values[i,1])-float(testing_user.values[i,1]))/float(training_user.values[i,1]))*100
        total_AE_in[training_user.values[i,0]]=mape_in
        mape_total_in=mape_total_in+mape_in
# calculacting accuracy:
mape_average_out=mape_total_out/i
mape_average_in=mape_total_in/i
# Visualization:
print(mape_average_in,'% error (MAPE) for income')
print(mape_average_out,'% error (MAPE) for expense')  
fig, axes = plt.subplots(1, 2)       
plt.rcdefaults()

bin_list=[]
for i in range(0,200,2):
    bin_list.append(i)
axes[0].hist(total_AE_in.values(), rwidth=0.7,bins=bin_list, color='g')
axes[1].hist(total_AE_out.values(), rwidth=0.7,bins=bin_list,  color='r')
axes[0].set_title('Income error percentage')
axes[1].set_title('Expense error percentage')
axes[1].set_ylabel('Number of Clients')
axes[0].set_ylabel('Number of Clients')
axes[1].set_xlabel('MAPE (error %)')
axes[0].set_xlabel('MAPE (error %)')
plt.grid(axis='y', alpha=0.75)   


1. Dividing users to highly accurate users versus low accurate users based on the above average technique (threshold is 10% MAPE)
2. Aggregating dataframe based on user id and month

In [1]:
#Dividing users to highly accurate users versus low accurate users:
Accurate_users={}
Not_accurate_users={}
threshold=10
for key in total_AE_in:
    value=total_AE_in[key]
    if value<=threshold:
        Accurate_users[key]=value
    else:
        Not_accurate_users[key]=value
for key in total_AE_out:
    value=total_AE_out[key]
    if value<=10:
        Accurate_users[key]=value
    else:
        Not_accurate_users[key]=value
cond = ~training['user_id'].isin(Accurate_users.keys())
accurate=training.drop(training[cond].index)
cond = training['user_id'].isin(Accurate_users.values())
not_accurate=training.drop(training[cond].index)
#Aggregated dataframe based on user id and month:
not_accurate_monthly=not_accurate.groupby(['user_id','month'], as_index=False).amount_n26_currency.agg(['max', 'min', 'count', 'median', 'mean','sum'])
not_accurate_monthly=not_accurate_monthly.reset_index()
accurate_monthly=accurate.groupby(['user_id', 'month'], as_index=False).amount_n26_currency.agg(['max', 'min', 'count', 'median', 'mean','sum'])
accurate_monthly=accurate_monthly.reset_index()

Histogram: Amount of transaction per month for highly accurate users versus low accurate users:

In [1]:
fig, axes = plt.subplots(1, 2)
bin_list=[]
for i in range(0,150,2):
    bin_list.append(i)
accurate_monthly['mean'].plot.hist(bins=bin_list,grid=True, rwidth=0.9,label='accurate',ax=axes[0], color='g')
not_accurate_monthly['mean'].plot.hist(bins=bin_list, grid=True, rwidth=0.9, label='not_accurate', ax=axes[1], color='r')
axes[1].set_xlabel('Amount of transactions')
axes[0].set_xlabel('Amount of transactions')
axes[1].set_ylabel('Number of Clients')
axes[0].set_ylabel('Number of Clients')
axes[1].legend()
axes[0].legend()
plt.legend(loc='upper right')
plt.grid(axis='y', alpha=0.75)

Comparing the input and output transactions between highly accurate users and low accurate users:

In [1]:
#Aggregated dataframe based on direction column:
not_accurate_direction=not_accurate.groupby(['direction'], as_index=False).amount_n26_currency.agg(['max', 'min', 'count', 'median', 'mean','sum'])
not_accurate_direction=not_accurate_direction.reset_index()
accurate_direction=accurate.groupby(['direction'], as_index=False).amount_n26_currency.agg(['max', 'min', 'count', 'median', 'mean','sum'])
accurate_direction=accurate_direction.reset_index()
#Visualization:
sns.set_style('whitegrid')
fig, ax =plt.subplots(1,2)
plt.subplots_adjust(hspace = 0.8)
fig.set_size_inches(11.7, 4.27)
sns.barplot(y='direction',palette = 'hls',order = ["In", "Out"] ,x='mean', data=accurate_direction, ax=ax[0], estimator=mean).set_title('Amount of Transaction Directions (High accuracy users)')
sns.barplot(y='direction',palette = 'hls',order = ["In", "Out"], x='mean', data=not_accurate_direction, ax=ax[1], estimator=mean).set_title('Amount of Transaction Directions (Low accuracy users)')

Comparing the reasons of transactions between highly accurate users and low accurate users:

In [1]:
#Aggregated dataframe based on month and explanation_y columns:
not_accurate_explanation_y=not_accurate.groupby(['month','explanation_y'], as_index=False).amount_n26_currency.agg(['max', 'min', 'count', 'median', 'mean','sum'])
not_accurate_explanation_y=not_accurate_explanation_y.reset_index()
accurate_explanation_y=accurate.groupby(['month','explanation_y'], as_index=False).amount_n26_currency.agg(['max', 'min', 'count', 'median', 'mean','sum'])
accurate_explanation_y=accurate_explanation_y.reset_index()
#visualization:
sns.set_style('whitegrid')
fig, ax =plt.subplots(1,2)
plt.subplots_adjust(hspace = 0.8)
fig.set_size_inches(11.7, 7.27)
plt.subplots_adjust(hspace = 0.2,wspace = 0.8)
sns.barplot(y='explanation_y',ci = None, x='mean', data=accurate_explanation_y, ax=ax[0], estimator=mean).set_title('Amount of Explanation_why (High accuracy users)')
sns.barplot(y='explanation_y',ci = None,  x='mean', data=not_accurate_explanation_y, ax=ax[1], estimator=mean).set_title('Amount of explanation_why (Low accuracy users)')

Comparing the transaction types between highly accurate users and low accurate users:

In [1]:
#Aggregated dataframe based on the explanation_x column:
not_accurate_explanation_x=not_accurate.groupby(['explanation_x'], as_index=False).amount_n26_currency.agg(['max', 'min', 'count', 'median', 'mean','sum'])
not_accurate_explanation_x=not_accurate_explanation_x.reset_index()
accurate_explanation_x=accurate.groupby(['explanation_x'], as_index=False).amount_n26_currency.agg(['max', 'min', 'count', 'median', 'mean','sum'])
accurate_explanation_x=accurate_explanation_x.reset_index()
sns.set_style('whitegrid')
#visualization:
fig, ax =plt.subplots(1,2)
plt.subplots_adjust(hspace = 0.8)
fig.set_size_inches(11.7, 7.27)
plt.subplots_adjust(hspace = 0.2,wspace = 0.8)
sns.barplot(y='explanation_x',ci = None, x='mean', data=accurate_explanation_x, ax=ax[0], estimator=mean).set_title('Amount of transaction types (accurate)')
sns.barplot(y='explanation_x',ci = None,  x='mean', data=not_accurate_explanation_x, ax=ax[1], estimator=mean).set_title('Amount of transaction types (not_accurate)')

Comparing the transaction media between highly accurate users and low accurate users:

In [1]:
#Aggregated dataframe based on month and agent columns:
not_accurate_agent=not_accurate.groupby(['month','agent'], as_index=False).amount_n26_currency.agg(['max', 'min', 'count', 'median', 'mean','sum'])
not_accurate_agent=not_accurate_agent.reset_index()
not_accurate_agent.to_csv('not_accurate_agent.csv')
accurate_agent=accurate.groupby(['month','agent'], as_index=False).amount_n26_currency.agg(['max', 'min', 'count', 'median', 'mean','sum'])
accurate_agent=accurate_agent.reset_index()
accurate_agent.to_csv('accurate_agent.csv')
#visualization:
sns.set_style('whitegrid')
fig, ax =plt.subplots(1,2)
plt.subplots_adjust(hspace = 0.8)
fig.set_size_inches(11.7, 7.27)
plt.subplots_adjust(hspace = 0.2,wspace = 0.8)
sns.barplot(y='agent',ci = None, x='mean', data=accurate_agent, ax=ax[0], estimator=mean).set_title('Amount of Transaction per Medium (accurate)')
sns.barplot(y='agent',ci = None,  x='mean', data=not_accurate_agent, ax=ax[1], estimator=mean).set_title('Amount of Transaction per Medium (not_accurate)')

Data pivoting:

In [1]:
#converting distinct entries of explanation_y values to columns:
pivoted_explanation_y=testing_training.pivot_table( index = ['user_id','month'] , columns=['explanation_y'], values = 'amount_n26_currency', aggfunc='mean').fillna(0)
pivoted_explanation_y_training=pivoted_explanation_y.reset_index()
#converting distinct entries of explanation_x values to columns:
pivoted_explanation_x=testing_training.pivot_table( index = ['user_id','month'] , columns=['explanation_x'], values = 'amount_n26_currency', aggfunc='mean').fillna(0)
pivoted_explanation_x=pivoted_explanation_x.reset_index()
#converting distinct entries of direction values to columns:
pivoted_direction=testing_training.pivot_table( index = ['user_id','month'] , columns=['direction'], values = 'amount_n26_currency', aggfunc='mean').fillna(0)
pivoted_direction=pivoted_direction.reset_index()
#converting distinct entries of direction values to columns:
pivoted_agent=testing_training.pivot_table( index = ['user_id','month'] , columns=['agent'], values = 'amount_n26_currency', aggfunc='mean').fillna(0)
pivoted_agent=pivoted_agent.reset_index()
# group by user_id and month columns:
monthly=testing_training.groupby(['user_id','month'], as_index=False).amount_n26_currency.mean()
monthly=monthly.reset_index()
# merging all pivoted data:
pivoted=pd.merge(left=monthly,right=pivoted_explanation_y,how='left', left_on=['user_id','month'], right_on=['user_id','month'])
pivoted=pd.merge(left=pivoted,right=pivoted_explanation_x,how='left', left_on=['user_id','month'], right_on=['user_id','month'])
pivoted=pd.merge(left=pivoted,right=pivoted_agent,how='left', left_on=['user_id','month'], right_on=['user_id','month'])
pivoted=pd.merge(left=pivoted,right=pivoted_direction,how='left', left_on=['user_id','month'], right_on=['user_id','month'])
preprocessed=pivoted.reset_index()
# dividing data to testing and training (month July is testing)
preprocessed_testing=preprocessed[preprocessed['month']==7]
preprocessed_training=preprocessed[preprocessed['month']!=7]
preprocessed.head()

Lasso model to predict July data:

In [1]:
# selecting training and testing columns:
columns=preprocessed_training.columns
columns=columns.drop('user_id')
columns=columns.drop('index')
columns=columns.drop('level_0')
columns=columns.drop('In')
columns=columns.drop('Out')
y_train = preprocessed_training[['In','Out']]
preprocessed_training=preprocessed_training[columns]
X_train=preprocessing.scale(preprocessed_training[columns])
print(X_train.shape)
X_test=preprocessing.scale(preprocessed_testing[columns])
y_test = preprocessed_testing[['In','Out']]
# defining the model:
model_july = Lasso(alpha=1.0, max_iter = 100000)
# training the model:
model_july.fit(X_train, y_train)
# predicting unseen values (testing is month july):
prediction_result=model_july.predict(X_test)
# instantiating accuracy variables:
total_AE_out={}
mape_total_out=0
total_AE_in={}
mape_total_in=0
# avoiding division by zero error using temporary values:
y_test.loc[y_test['In'] ==0, 'In'] =0.46395169417715465
y_test.loc[y_test['Out'] ==0, 'Out'] = 0.6283420872431762
# measuring accuracy (MAPE):
for i in range(0, len(y_test)):
        mape_out=abs((y_test.iloc[i]['Out']-prediction_result[i][1])/y_test.iloc[i]['Out'])*100
        total_AE_out[i]=mape_out
        mape_total_out=mape_total_out+mape_out
        mape_in=abs((y_test.iloc[i]['In']-prediction_result[i][0])/y_test.iloc[i]['In'])*100
        total_AE_in[i]=mape_in
        mape_total_in=mape_total_in+mape_in
mape_average_in=mape_total_in/i
mape_average_out=mape_total_out/i
#visualization:
print(mape_average_in,'% error for incomes')
print(mape_average_out,'% error for expenses')
fig, axes = plt.subplots(1, 2, constrained_layout=True)       
plt.rcdefaults()
axes[0].hist(total_AE_in.values(), color='g',bins=bin_list)
axes[1].hist(total_AE_out.values(), color='r',bins=bin_list)
axes[0].set_title('Income error percentage')
axes[1].set_title('Expense error percentage')
axes[1].set_ylabel('Number of Clients')
axes[0].set_ylabel('Number of Clients')
axes[1].set_xlabel('MAPE (error %)')
axes[0].set_xlabel('MAPE (error %)')
plt.grid(axis='y', alpha=0.75)

shrinked features:

In [1]:
print(model_july.coef_[0])
print(preprocessed_training.columns[(model_july.coef_[0]!=0).ravel().tolist()])
print(model_july.coef_[1])
print(preprocessed_training.columns[(model_july.coef_[1]!=0).ravel().tolist()])

Lasso model to generate result for the month August:

In [1]:
columns=preprocessed.columns

preprocessed_selected=preprocessed[columns]
y_train =preprocessed_selected[['In','Out']]

y_test =preprocessed_selected[['In','Out']]
columns=columns.drop('user_id')
columns=columns.drop('index')
columns=columns.drop('In')
columns=columns.drop('Out')
preprocessed_selected=preprocessed_selected[columns]
X_train=preprocessing.scale(preprocessed_selected[columns])
preprocessed_selected['month']=8
X_test=preprocessing.scale(preprocessed_selected[columns])
model = Lasso(alpha=1.0, max_iter = 100000)
model.fit(X_train, y_train)
prediction_result=model.predict(X_test)
preprocessed['Income'] = prediction_result[:, 0]
preprocessed['Expense'] =prediction_result[:, 1]
preprocessed.to_csv("result.csv", columns=['user_id','month','Income', 'Expense'], index=False)